<a href="https://colab.research.google.com/github/yogesh-bhattarai/GEN_AI/blob/main/exercise_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [3]:
df= pd.read_csv("/content/ExerciseRecommendation.csv")

In [4]:
df.head(10)

,exercise_id,fitness_goal,exercise_name,exercise_type,equipment_needed,duration_or_reps
0,1,Weight Loss,Jump rope,Cardio,Jump rope,3 sets of 15 reps
1,2,Weight Loss,Burpees,Cardio,NaN,3 sets of 15 reps
2,3,Weight Loss,HIIT,Cardio,Mat,3 sets of 15 reps
3,4,Weight Loss,Running,Cardio,Treadmill / Outdoor,20-40 mins
4,5,Weight Loss,Cycling,Cardio,Stationary bike,20-40 mins
5,6,Weight Loss,Mountain climbers,Cardio,Mat,3 sets of 15 reps
6,7,Weight Loss,Squats,Strength,None / Barbell,3 sets of 8-12 reps
7,8,Muscle Gain,Bench press,Strength,Barbell,3 sets of 8-12 reps
8,9,Muscle Gain,Deadlifts,Strength,Barbell,3 sets of 8-12 reps
9,10,Muscle Gain,Squats,Strength,None / Barbell,3 sets of 8-12 reps


In [10]:
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class ExerciseRecommender:
    def __init__(self, data):
        self.df = pd.DataFrame(data)
        self.vectorizer = TfidfVectorizer()
        self._prepare_data()

    def _prepare_data(self):
        # Combine columns into a single string for vectorization
        # Handle NaN values by filling them with empty strings
        self.df['combined'] = (
            self.df['fitness_goal'].fillna('') + ' ' +
            self.df['exercise_type'].fillna('') + ' ' +
            self.df['equipment_needed'].fillna('') + ' ' +
            self.df['duration_or_reps'].fillna('')
        )
        self.tfidf_matrix = self.vectorizer.fit_transform(self.df['combined'])

    def recommend(self, fitness_goal, exercise_type, top_n=5):
      user_input = f"{fitness_goal} {exercise_type}"
      user_vec = self.vectorizer.transform([user_input])
      similarity = cosine_similarity(user_vec, self.tfidf_matrix).flatten()

      # Create a result DataFrame with scores
      self.df['similarity_score'] = similarity
      top_results = self.df.sort_values(by='similarity_score', ascending=False)

      # Drop duplicates by exercise_name
      unique_results = top_results.drop_duplicates(subset='exercise_name').head(top_n)

      return unique_results[[
        'exercise_name', 'fitness_goal', 'exercise_type', 'similarity_score'
    ]]


# --- Sample Data ---
df= pd.read_csv("/content/ExerciseRecommendation.csv")
# --- Create object and save ---
recommender = ExerciseRecommender(df)

with open("exercise_recommender.pkl", "wb") as f:
    pickle.dump(recommender, f)

print("Recommender object saved as 'exercise_recommender.pkl'")

Recommender object saved as 'exercise_recommender.pkl'


In [12]:
import pickle

# Load the recommender object
with open("exercise_recommender.pkl", "rb") as f:
    recommender = pickle.load(f)

# Recommend exercises
result = recommender.recommend("weight loss", "cardio")
print(result)


           exercise_name fitness_goal exercise_type  similarity_score
11957            Burpees  Weight Loss        Cardio          0.859842
11958               HIIT  Weight Loss        Cardio          0.784249
11961  Mountain climbers  Weight Loss        Cardio          0.784249
1470           Jump rope  Weight Loss        Cardio          0.587208
3                Running  Weight Loss        Cardio          0.558752


['jump rope']



Recommendations for 'Deadlifts' based on fitness_goal, equipment_needed, duration_or_reps:
1. Deadlifts (Type: Strength, Similarity: 1.000)
2. Bench press (Type: Strength, Similarity: 1.000)
